# Test Question Generation for Perplexity API

This notebook demonstrates how to:
1. Load patient data
2. Find discharge medications
3. Generate templated questions for Perplexity API

In [ ]:
import sys
import pandas as pd
import json
from utils.Dataset import MIMICDataset
from example_question_generator import QuestionGenerator

## Step 1: Load Patient Data

In [ ]:
# Initialize dataset - load just a few patients to test quickly
print("Loading patient data...")
dataset = MIMICDataset(
    data_dir="D:/Datathon/physionet.org/files/mimiciv/3.1",  # Update this path if needed
    max_patients=5  # Just load 5 patients to test quickly
)

# Get all patients
patients = dataset.get_all_patients()
print(f"\nLoaded {len(patients)} patients")

## Step 2: Find Discharge Medication with Longest Duration

In [ ]:
# Find a patient with discharge medications and calculate duration
best_patient = None
best_hadm_id = None
best_medication = None
longest_duration = pd.Timedelta(0)

for patient in patients:
    for admission in patient.admissions:
        hadm_id = admission.hadm_id
        
        # Get discharge medications
        discharge_meds = dataset.get_discharge_medications(hadm_id)
        
        if discharge_meds is not None and not discharge_meds.empty:
            # Calculate duration for each medication
            discharge_meds['duration'] = pd.to_datetime(discharge_meds['stoptime']) - pd.to_datetime(discharge_meds['starttime'])
            
            # Find the medication with longest duration
            for _, med in discharge_meds.iterrows():
                if med['duration'] > longest_duration:
                    longest_duration = med['duration']
                    best_medication = med
                    best_hadm_id = hadm_id
                    best_patient = patient

if best_medication is not None:
    print("="*80)
    print("FOUND DISCHARGE MEDICATION WITH LONGEST DURATION")
    print("="*80)
    print(f"\nPatient ID: {best_patient.subject_id}")
    print(f"Admission ID: {best_hadm_id}")
    print(f"\nMedication: {best_medication['drug']}")
    print(f"Dose: {best_medication['dose_val_rx']} {best_medication['dose_unit_rx']}")
    print(f"Route: {best_medication['route']}")
    print(f"Duration: {longest_duration}")
    print(f"Start: {best_medication['starttime']}")
    print(f"Stop: {best_medication['stoptime']}")
else:
    print("\nNo discharge medications found in the sample patients.")
    print("Try increasing max_patients parameter.")

## Step 3: Generate Question Using Template

In [ ]:
# Initialize the question generator
generator = QuestionGenerator('perplexity_question_templates.json')

# Generate the side effects question
question = generator.generate_question(
    category='medications',
    template_id='med_side_effects',
    medication_name=best_medication['drug']
)

print("="*80)
print("GENERATED QUESTION FOR PERPLEXITY API")
print("="*80)
print(f"\nQuestion: {question}")

## Step 4: Also Generate 'How to Take' Question with Dosing Info

In [ ]:
how_to_take = generator.generate_question(
    category='medications',
    template_id='med_how_to_take',
    medication_name=best_medication['drug'],
    dose_val=best_medication['dose_val_rx'],
    dose_unit=best_medication['dose_unit_rx'],
    route=best_medication['route']
)

print(f"\nHow to Take Question: {how_to_take}")

## Step 5: Format for Perplexity API Call

In [ ]:
# Create the payload structure for Perplexity API
perplexity_payload = {
    "model": "sonar",
    "messages": [
        {
            "role": "user",
            "content": question
        }
    ]
}

print("="*80)
print("PERPLEXITY API PAYLOAD FORMAT")
print("="*80)
print(json.dumps(perplexity_payload, indent=2))

## Step 6: (Optional) Actually Call Perplexity API

In [ ]:
# Uncomment below to actually make the API call

# import os
# from perplexity import Perplexity

# # Set your API key (replace with your actual key or use environment variable)
# os.environ["PERPLEXITY_API_KEY"] = "your-api-key-here"

# # Initialize the client
# client = Perplexity()

# # Make the API call
# completion = client.chat.completions.create(
#     model="sonar",
#     messages=[
#         {"role": "user", "content": question}
#     ]
# )

# # Print the AI's response
# print("="*80)
# print("PERPLEXITY API RESPONSE")
# print("="*80)
# print(completion.choices[0].message.content)

## Bonus: View All Available Question Templates

In [ ]:
# See all available templates
all_templates = generator.get_all_templates()

print(f"\nTotal available templates: {len(all_templates)}\n")

# Show medication templates
print("Medication Question Templates:")
med_templates = generator.get_all_templates('medications')
for t in med_templates:
    print(f"  - {t['id']}: {t['template']}")